In [1]:
#Basic Libraries
import re
import numpy as np
import pandas as pd
from pprint import pprint
import sys
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy for lemmatization
import spacy
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
#Load Dataset
df=pd.read_csv('/home/smriti/Desktop/NLP/MITACS/Anxiety/Data/CSV/Reddit/rAnxiety14.csv')
#getting rid of NaN
df=df.replace(np.nan, '', regex=True)
#getting rid of deleted values
df['Text']=df['Text'].replace('[deleted]','')
#Combining title and text
df["Post"] = df["Title"] + df["Text"]
#Now that we don't need Title or Text, we drop those columns before saving the file
df=df.drop(['Title', 'Text'], axis = 1)

/home/smriti/.local/lib/python3.5/site-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [4]:
df=df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [5]:
df.head()

,Number of Comments,Date Posted,Post
0,66,2014-12-26 00:39:39 EST-0500,How I feel drinking coffee during a panic attack
1,111,2014-12-24 19:16:42 EST-0500,Merry X-mas to us loners.Many of us who suffer...
2,74,2014-12-22 16:24:53 EST-0500,Does anybody here suffers from always feel gui...
3,18,2014-12-25 14:16:04 EST-0500,"Hey you (yes, you!)Whatever today holds, I hop..."
4,55,2014-12-28 13:57:32 EST-0500,[RANT] I always feel like everyone secretly ha...


In [6]:
# Convert to list
data=df.Post.values.tolist()
# Remove new line characters
data=[re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data=[re.sub("\'", "", sent) for sent in data]

In [7]:
#Function to clean up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) 

In [8]:
#Clean up text
data_words=list(sent_to_words(data))
print(data_words[:1])

[['how', 'feel', 'drinking', 'coffee', 'during', 'panic', 'attack']]


In [9]:
# Build the bigram and trigram models
bigram=gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram=gensim.models.Phrases(bigram[data_words], threshold=100)

In [10]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod=gensim.models.phrases.Phraser(bigram)
trigram_mod=gensim.models.phrases.Phraser(trigram)

In [11]:
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['how', 'feel', 'drinking', 'coffee', 'during', 'panic_attack']


In [12]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


In [13]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

# Create Dictionary needed for topic modelling
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=60,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

[['feel', 'drink', 'coffee']]


In [15]:
# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(44,
  '0.000*"circus" + 0.000*"waist" + 0.000*"battery" + 0.000*"appearance" + '
  '0.000*"warped" + 0.000*"strangeness" + 0.000*"scatter" + 0.000*"odd" + '
  '0.000*"distorted" + 0.000*"demente"'),
 (35,
  '0.000*"circus" + 0.000*"waist" + 0.000*"battery" + 0.000*"appearance" + '
  '0.000*"warped" + 0.000*"strangeness" + 0.000*"scatter" + 0.000*"odd" + '
  '0.000*"distorted" + 0.000*"demente"'),
 (2,
  '0.000*"circus" + 0.000*"waist" + 0.000*"battery" + 0.000*"appearance" + '
  '0.000*"warped" + 0.000*"strangeness" + 0.000*"scatter" + 0.000*"odd" + '
  '0.000*"distorted" + 0.000*"demente"'),
 (11,
  '0.000*"circus" + 0.000*"waist" + 0.000*"battery" + 0.000*"appearance" + '
  '0.000*"warped" + 0.000*"strangeness" + 0.000*"scatter" + 0.000*"odd" + '
  '0.000*"distorted" + 0.000*"demente"'),
 (39,
  '0.000*"circus" + 0.000*"waist" + 0.000*"battery" + 0.000*"appearance" + '
  '0.000*"warped" + 0.000*"strangeness" + 0.000*"scatter" + 0.000*"odd" + '
  '0.000*"distorted" + 0.000*"demente"

In [16]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -17.13593795846216


In [17]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3861782080953099


In [18]:
num_topics = 60
topic_words = []
for i in range(num_topics):
    tt = lda_model.get_topic_terms(i,10)
    topic_words.append([id2word[pair[0]] for pair in tt])

In [19]:
for i in range(0,60):
    print(topic_words[i])

['room', 'mother', 'treat', 'exactly', 'stranger', 'tl_dr', 'childhood', 'awake', 'rate', 'nightmare']
['circus', 'waist', 'battery', 'appearance', 'warped', 'strangeness', 'scatter', 'odd', 'distorted', 'demente']
['circus', 'waist', 'battery', 'appearance', 'warped', 'strangeness', 'scatter', 'odd', 'distorted', 'demente']
['depression', 'tonight', 'suffer', 'anxiety', 'different', 'strangeness', 'distorted', 'odd', 'scatter', 'dicke']
['anxiety', 'remember', 'try', 'see', 'year', 'thing', 'look', 'deal', 'could', 'say']
['circus', 'waist', 'battery', 'appearance', 'warped', 'strangeness', 'scatter', 'odd', 'distorted', 'demente']
['anger', 'fully', 'regular', 'maintain', 'powerful', 'competition', 'suspect', 'previous', 'fear', 'scatter']
['feel', 'be', 'anxiety', 'really', 'make', 'want', 'know', 'have', 'thing', 'think']
['tomorrow', 'teacher', 'appointment', 'cancel', 'session', 'piss', 'snap', 'conscious', 'finally', 'therapy']
['reality', 'refuse', 'psych', 'drug', 'speak', 'co

In [20]:
df=pd.DataFrame({'Year':[],'Source':[],'Topic_ID':[],'Most_freq_words':[]})
df.head()

,Most_freq_words,Source,Topic_ID,Year


In [21]:
df['Most_freq_words']=topic_words

In [22]:
df = df.assign(Year='2014')
df = df.assign(Source='Reddit')

In [23]:
ls=[]
for i in range(0,60):
    ls.append(i)
df['Topic_ID']=ls

In [24]:
df.head()

,Most_freq_words,Source,Topic_ID,Year
0,"[room, mother, treat, exactly, stranger, tl_dr...",Reddit,0,2014
1,"[circus, waist, battery, appearance, warped, s...",Reddit,1,2014
2,"[circus, waist, battery, appearance, warped, s...",Reddit,2,2014
3,"[depression, tonight, suffer, anxiety, differe...",Reddit,3,2014
4,"[anxiety, remember, try, see, year, thing, loo...",Reddit,4,2014


In [25]:
df.to_csv("topic_words_r2014.csv")